In [ ]:
### Set up the root folder
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/GPT

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/GPT


In [ ]:
!pip install -r ./requirements.txt -qq
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 879.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import transformers
import pandas as pd
from model import GPT2PPLV2 as GPT2PPL
from sklearn.metrics import accuracy_score
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
datasetname = [ 'DagPap22','chatgpt_human_combined','IELTS_GPT35_Human_Comparison']

detectgptmodelname = 'detectgpt'
gptzeromodelname = 'gptzero'

def getInputfileName(datasetname):
    return f"./output/{datasetname}_Corpus.csv"

def getOutputfileName(datasetname, modelname):
    return f"./output/{datasetname}_result_{modelname}.csv"

In [ ]:
from sklearn.metrics import accuracy_score, classification_report### DetectGPT
#function to process the detectGPT and Chat GPT model on a given datasets file
def processGPTModel(datasetFile):
    print(f"Processing for datasets : {datasetFile}")
    df_combined = pd.read_csv(getInputfileName(datasetFile))
    # initialize the model
    model = GPT2PPL()

    sentence = """Some people believe that a college or a university should be available to all students. Others think that higher education should be available only to good students. In my opinion, higher education should be available to all students. Why do I think so?
    Firstly, to get higher education is a right of any person. If higher education is available only to good student, it will be infringement of human rights. We cannot say to man that she or he is not entitled to get higher education because it will be anti-constitutional. On the other hand, a college or university can have own rules for students who don't have a good GPA. For example, students can graduate from college if they have only "A" or "B". Another example, students can take several times the same exam while they won’t receive a good grade. It is very important to give a right to get higher education to all students, because it will be their chance to change something in their life, get better job in the future or built their career.
    Secondly, who gives us right to share people in good and bad students. Maybe they have good abilities for studying, yet, these students chose the wrong way ignoring learning when they were high school students. We have a lot of examples when people didn't have a good grade when they were a high school students, however, they could graduate from college and became to work successfully. We know many examples about great people, Pushkin, Mendeleev, who weren't good students in all subjects, but they were very talented persons and whole world knows their names. To be a good or bad students are not the same to have abilities for learning. We should understand and remember it.
    Some people can say that government and parents just waste money for education bad students. Nonetheless, the government also can change rules for those students. For example, not support financially if the student has a grade "C" or worse. However, our government sustains any students. I think that it is the right way, because we live in a free country and any man has the same right to get higher education. In summary, in spite of colleges and universities are mostly interested in good students, I strongly believe that any students, good or bad, should have the same right to earn high education."""

    temp = model(sentence, 200, "v1.1")

    df_combined['token_count']= [len(tokenize.word_tokenize(x)) for x in df_combined.text]
    result_detectgpt = pd.DataFrame(columns=['sr_no','prob', 'label', 'desc'])
    result_gptzero = pd.DataFrame(columns=['sr_no','Perplexity', 'Perplexity per line', 'Burstiness', 'label', 'desc'])

    #“perplexity” (aka randomness) and “burstiness” (aka variance)
    # GPTZero:
    # To determine whether an excerpt is written by a bot, GPTZero uses two indicators: "perplexity" and "burstiness."

    # Perplexity
    # Perplexity measures the complexity of text; if GPTZero is perplexed by the text, then it has a high complexity and it's more likely to be human-written.
    # However, if the text is more familiar to the bot — because it's been trained on such data — then it will have low complexity and therefore is more likely to be AI-generated.

    # “burstiness”
    # If a term is used once in a document, then it is likely to be used again. This phenomenon is called burstiness, and it implies that the second and later appearances of a word are less significant than the first appearance

    for i in range(len(df_combined)):
      print(i)
      try:
        token_count = min(df_combined['token_count'][i], 200)
        pred, desc = model(df_combined.text[i], token_count, "v1.1")
        pred['desc']= desc
        pred['sr_no']= df_combined.sr_no[i]
      except:
        pred['prob']= None
        pred['label']= None
        pred['desc']= 'error'
        pred['sr_no']= df_combined.sr_no[i]
      result_detectgpt = result_detectgpt.append(pred, ignore_index=True)

      try:
        pred, desc = model(df_combined.text[i], token_count, "v1")
        pred = dict(pred)
        pred['desc']= desc
        pred['sr_no']= df_combined.sr_no[i]
      except:
        pred['Perplexity']= None
        pred['Perplexity per line']= None
        pred['Burstiness']= None
        pred['label']= None
        pred['desc']= 'error'
        pred['sr_no']= df_combined.sr_no[i]
      result_gptzero = result_gptzero.append(pred, ignore_index=True)

    result_detectgpt.to_csv(getOutputfileName(datasetFile, detectgptmodelname))
    result_gptzero.to_csv(getOutputfileName(datasetFile, gptzeromodelname))

    result_detectgpt['label']=result_gptzero.label.map({1:'human', 0:'chatgpt'})
    result_gptzero['label']=result_gptzero.label.map({1:'human', 0:'chatgpt'})

    gptzero_final=pd.concat([result_gptzero, df_combined], axis=1)
    gptzero_final = gptzero_final[['sr_no', 'source', 'label']].dropna()
    detectgpt_final=pd.concat([result_gptzero, df_combined], axis=1)
    detectgpt_final = detectgpt_final[['sr_no', 'source', 'label']].dropna()

    print("### DetectGPT")
    print(classification_report(detectgpt_final['source'], detectgpt_final['label']))
    print("### GPTZero")
    print(classification_report(gptzero_final['source'], gptzero_final['label']))

In [ ]:
#Process for  all datasets one by one
for datasetfile in datasetname:
    processGPTModel(datasetfile)


Processing for datasets : DagPap22


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


0


<ipython-input-5-4a5a353a9b78>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_detectgpt = result_detectgpt.append(pred, ignore_index=True)
<ipython-input-5-4a5a353a9b78>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_gptzero = result_gptzero.append(pred, ignore_index=True)


1


...

<ipython-input-5-4a5a353a9b78>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_detectgpt = result_detectgpt.append(pred, ignore_index=True)
<ipython-input-5-4a5a353a9b78>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_gptzero = result_gptzero.append(pred, ignore_index=True)


182


<ipython-input-5-4a5a353a9b78>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_detectgpt = result_detectgpt.append(pred, ignore_index=True)


### DetectGPT
              precision    recall  f1-score   support

     chatgpt       0.00      0.00      0.00         0
       human       1.00      0.89      0.94       107

    accuracy                           0.89       107
   macro avg       0.50      0.44      0.47       107
weighted avg       1.00      0.89      0.94       107

### GPTZero
              precision    recall  f1-score   support

     chatgpt       0.00      0.00      0.00         0
       human       1.00      0.89      0.94       107

    accuracy                           0.89       107
   macro avg       0.50      0.44      0.47       107
weighted avg       1.00      0.89      0.94       107



<ipython-input-5-4a5a353a9b78>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_gptzero = result_gptzero.append(pred, ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0